##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 단일 작업자 멀티 GPU 훈련 마이그레이션하기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/migrate/mirrored_strategy"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a> </td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/guide/migrate/mirrored_strategy.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a>   </td>
</table>

이 가이드는 단일 작업자 멀티 GPU 워크플로를 TensorFlow 1에서 TensorFlow 2로 마이그레이션하는 방법을 설명합니다.

다음과 같이 한 머신에서 멀티 GPU로 동기식 훈련을 수행할 수 있습니다.

- TensorFlow 1에서는 `tf.distribute.MirroredStrategy`와 함께 `tf.estimator.Estimator` API를 사용합니다.
- TensorFlow 2에서는 `tf.distribute.MirroredStrategy`와 함께 [Keras Model.fit](https://www.tensorflow.org/tutorials/distribute/keras) 또는 [사용자 정의 훈련 루프](https://www.tensorflow.org/tutorials/distribute/custom_training)를 사용할 수 있습니다. [TensorFlow를 사용하는 분산 훈련](https://www.tensorflow.org/guide/distributed_training#mirroredstrategy) 가이드에서 자세히 알아보세요.

## 설치하기

데모를 위해 가져오기 및 간단한 데이터세트로 시작해 보겠습니다.

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]
eval_features = [[4., 4.5], [5., 5.5], [6., 6.5]]
eval_labels = [[0.8], [0.9], [1.]]

## TensorFlow 1: tf.estimator.Estimator로 단일 작업자 분산 훈련하기

이 예제는 단일 작업자 멀티 GPU 훈련의 TensorFlow 1 정식 워크플로를 보여줍니다. `tf.estimator.Estimator`의 `config` 매개변수를 통해 배포 전략(`tf.distribute.MirroredStrategy`)을 설정해야 합니다.

In [ ]:
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

def _eval_input_fn():
  return tf1.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

def _model_fn(features, labels, mode):
  logits = tf1.layers.Dense(1)(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())
  return tf1.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

strategy = tf1.distribute.MirroredStrategy()
config = tf1.estimator.RunConfig(
    train_distribute=strategy, eval_distribute=strategy)
estimator = tf1.estimator.Estimator(model_fn=_model_fn, config=config)

train_spec = tf1.estimator.TrainSpec(input_fn=_input_fn)
eval_spec = tf1.estimator.EvalSpec(input_fn=_eval_input_fn)
tf1.estimator.train_and_evaluate(estimator, train_spec, eval_spec)

## TensorFlow 2: Keras로 단일 작업자 훈련하기

TensorFlow 2로 마이그레이션하는 경우 `tf.distribute.MirroredStrategy`와 함께 Keras API를 사용할 수 있습니다.

모델 빌드에 `tf.keras` API를 사용하고 훈련에 Keras `Model.fit`을 사용하는 경우 주요 차이점은 `tf.estimator.Estimator`에 `config`를 정의하는 대신 `Strategy.scope` 컨텍스트에서 Keras 모델, 옵티마이저 및 메트릭을 인스턴스화하는 것입니다.

사용자 정의 훈련 루프를 사용해야 하는 경우 [사용자 정의 훈련 루프와 함께 tf.distribute.Strategy 사용하기](https://www.tensorflow.org/guide/distributed_training#using_tfdistributestrategy_with_custom_training_loops) 가이드를 확인하세요.

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
eval_dataset = tf.data.Dataset.from_tensor_slices(
      (eval_features, eval_labels)).batch(1)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
  optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)

model.compile(optimizer=optimizer, loss='mse')
model.fit(dataset)
model.evaluate(eval_dataset, return_dict=True)

## 다음 단계

TensorFlow 2에서 `tf.distribute.MirroredStrategy`를 사용하는 분산 훈련에 대한 자세한 내용은 다음 설명서를 확인하세요

- [Keras를 사용하여 하나의 머신에서 분산 훈련하기](../../tutorials/distribute/keras) 튜토리얼
- [사용자 정의 훈련 루프로 하나의 머신에서 분산 훈련하기](../../tutorials/distribute/custom_training) 튜토리얼
- [TensorFlow를 사용하여 분산 훈련하기](../../guide/distributed_training) 가이드
- [멀티 GPU 사용하기](../../guide/gpu#using_multiple_gpus) 가이드
- [멀티 GPU 단일 호스트에서 성능 최적화하기(TensorFlow 프로파일러 사용)](../../guide/gpu_performance_analysis#2_optimize_the_performance_on_the_multi-gpu_single_host) 가이드